### indexTree
1. 单点更新某个值
2. 返回L...R范围的累加和
3. 线段树可以满足以上并进行范围更新，但index的优势在于

In [ ]:
'''
数组下标
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16
累加和范围
1,1-2,3,1-4,5,5-6,7,1-8,9,9-10,11,9-12,13,13-14,15,1-16

index = 010111000
的累加和范围为 010110001 ~ 010111000 (把最后一个1拆开+1 到index)

若想知道1-i的累加和，
i的二进制位01011100，则累加和 = [01011100](本身) + [01011000](去掉一个1) + [01010000](去掉一个1) + [01000000](去掉一个1..直到没有1)
'''

In [1]:
class IndexTree:
    def __init__(self,size):
        self.N = size
        #下标从1开始
        self.tree = [0] * (N+1)
        
    def sum_(self, index):
        ret = 0
        while index > 0:
            ret += self.tree[index]
            index -= index & (-index)
        return ret
    
    def add_(self, index, d): #调整受牵连的位置
        while index <= N:
            self.tree[index] += d 
            index += index & (-index)

### AC自动机
用于检索一篇文章，是否出现过某些敏感词汇。
* 思路类似KMP算法，通过将敏感词汇构建成前缀树，树节点的fail指针类似kmp的nexts数组，帮助文章匹配失败时快速遍历所有敏感词汇。
* 首先构建前缀树
* 然后遍历前缀树，将fail指针指向填好。头节点fail指向空(类似nexts[0] = -1)，所有头节点的子节点fail指向头节点(类似nexts[0] = 0)
* 检查文章时，当某一敏感词X的途中匹配失败(字符a失败)，通过fail指针，跳转到【其他敏感词从头开始到、必须以a结束的最长敏感词】

In [ ]:
class Node:
    def __init__(self, end=None, endUse=False, fail=None, nexts=None):
        self.end = end #如果一个node，end为空，不是结尾
        self.endUse = endUse #只有end不为空才有意义，表示这个字符串是否加入过答案(含有end敏感词)
        self.fail = fail
        if nexts is None:
            self.nexts = [None] * 26 #存储下一个字符，可能包含26种小写字符
        else:
            self.nexts = nexts
from collections import deque
class ACAutomation:
    def __init__(self):
        self.root = Node() #字符放在nexts的路径上，不会在节点上
        
    def insert(self, s): #只构建敏感词前缀树，fail指针由build填加
        string = list(s)
        cur = self.root
        index = 0
        for i in range(len(string)):
            index = ord(string[i]) - ord('a')
            if cur.nexts[index] is None:
                cur.nexts[index] = Node()
            cur = cur.nexts[index]
        cur.end = s
        return
    
    def build(self):
        #宽度优先遍历，添加fail指针
        queue = deque()
        queue.append(self.root)
        cur = None
        cfail = None
        while queue:
            cur = queue.popleft() #当前节点弹出，将子节点的fail指针填好（而不是设置弹出点的，因为找不到父了就）
            for i in range(26):#所有的路
                if cur.nexts[i] is not None:  #i号儿子是有效路径
                    cur.nexts[i].fail = self.root #默认连向头节点
                    cfail = cur.fail #子节点的fail初始化完成，查看父(cur)的fail是否非空，即能否跳转
                    while cfail is not None:
                        if cfail.nexts[i] is not None: #父的fail也有i号=i号儿子，可以跳连
                            cur.nexts[i].fail = cfail.nexts[i]
                            break
                        cfail = cfail.fail #如果父的fail没有i号儿子，就继续往父的fail的fail找（加速过程，类似nexts跳转
                    queue.append(cur.nexts[i]) #当前节点的所有后代加入到队列中
                    
    def containsWords(self, content): #content大文章
        string = list(content)
        cur = self.root
        follow = None
        ans = list() #包含敏感词列表
        for i in range(len(string)):
            index = ord(string[i]) - ord('a') #当前大文章的路
            #当前字符在这条路上没配出来，就随着fail方向走向下一条路径。即匹配失败时延fail跳转
            while cur.nexts[index] is None and cur != self.root:
                cur = cur.fail
            #两种情况：1下面有路了，2已经跳回root
            if cur.nexts[index] is not None:
                cur = cur.nexts[index]  #匹配成功，也延fail看一圈（失败是直接跳过去）
            else:
                cur = self.root
            follow = cur
            #cur不往下跳，由follow延fail方向过一遍，检查是否存在end命中
            while follow != self.root:
                if follow.endUse:
                    break
                if follow.end is not None:
                    ans.append(follow.end)
                    follow.endUse = True
                follow = follow.fail
        return ans
                
                